# Simulating Many Sources in `castor_etc`

Isaac Cheng - April 2022

This notebook shows how to process many point sources using the `castor_etc` Python
package. Note that we do not yet support generating multiple sources in the same aperture
(but you can always simulate your own and [use your own data](./custom_source.ipynb) with
the ETC!).


In [1]:
import astropy.units as u
import numpy as np
import pandas as pd
from IPython.core.display import display

from castor_etc.background import Background
from castor_etc.photometry import Photometry
from castor_etc.sources import PointSource
from castor_etc.telescope import Telescope


We will use the same `Telescope` and (sky) `Background` objects for all of our photometry
calculations.


In [2]:
MyTelescope = Telescope()
MyBackground = Background()
MyBackground.add_geocoronal_emission("avg")


Now we will generate many different point sources at different AB magnitudes, then find
the time required (in seconds) to achieve a signal-to-noise ratio of 5 and 10 in each
passband.


In [3]:
TARGET_SNRS = [5, 10]
AB_MAGS = np.arange(22, 29, 0.5)
REDDENING = 0.0

wavelengths = np.arange(90, 1200) * u.nm  # generate the spectrum over these wavelengths

# Make some dictionaries to store the results of our calculations
times_to_one_snr = dict.fromkeys(AB_MAGS)
times_to_reach_snrs = {snr: times_to_one_snr.copy() for snr in TARGET_SNRS}

for ab_mag in AB_MAGS:
    #
    # Initialize a point source and generate a flat spectrum in units of AB magnitude
    #
    MySource = PointSource()
    MySource.generate_uniform(wavelengths=wavelengths, value=ab_mag, unit="ABmag")
    #
    # Initialize a Photometry object and use an optimal aperture for a point source
    #
    MyPhot = Photometry(MyTelescope, MySource, MyBackground)
    MyPhot.use_optimal_aperture(quiet=True)  # don't print encircled energy fraction
    for snr in TARGET_SNRS:
        #
        # Find the time, in seconds, required to reach the specified signal-to-noise ratio
        #
        times_to_reach_snrs[snr][ab_mag] = MyPhot.calc_snr_or_t(
            snr=snr, reddening=REDDENING
        )


In [4]:
for snr in TARGET_SNRS:
    print(f"Times (s) required to reach S/N = {snr}")
    display(pd.DataFrame(times_to_reach_snrs[snr]).T)


Times (s) required to reach S/N = 5


,uv,u,g
22.0,4.957908,4.593260,3.779018
22.5,7.863590,7.293252,6.039466
23.0,12.477614,11.592785,9.698436
23.5,19.812547,18.458275,15.691195
24.0,31.493464,29.468363,25.682353
24.5,50.147026,47.243975,42.781496
25.0,80.065199,76.241466,73.147130
25.5,128.376954,124.297443,129.774100
26.0,207.211699,205.827268,241.765618
26.5,337.918757,348.870584,477.395276


Times (s) required to reach S/N = 10


,uv,u,g
22.0,15.676781,14.536679,12.022184
22.5,24.873186,23.101551,19.286615
23.0,39.489591,36.770523,31.153713
23.5,62.758298,58.672263,50.855618
24.0,99.896355,93.983071,84.345369
24.5,159.409359,151.456523,143.177495
25.0,255.375532,246.357669,251.090546
25.5,411.616818,406.414427,459.600175
26.0,669.709967,684.583921,885.788886
26.5,1105.256787,1187.849839,1804.992244
